In [ ]:
import polars as pl
import sys
import os

sys.path.append(os.path.dirname(os.path.abspath('')))

from libraries.client_stashapp import get_stashapp_client, StashAppClient

stash = get_stashapp_client()
stash_client = StashAppClient()

In [ ]:
stash_tags = stash_client.get_tags_by_names(["AI_Tagged", "AI_Reviewed", "Marker Status: Confirmed", "Marker Source: PySceneDetect", "Marker Source: Skier AI", "Marker Source: Manual", "Marker Source: timestamp.trade", "Marker Source: TPDB"])

In [ ]:
confirmed_markers = stash.find_scene_markers(
    {
        "tags": {
            "value": [stash_tags.marker_status_confirmed["id"]],
            "modifier": "INCLUDES_ALL",
        }
    },
    fragment="id primary_tag { id name } tags { id name } scene { id title }",
)
scene_ids_of_confirmed_markers = [confirmed_markers["scene"]["id"] for confirmed_markers in confirmed_markers]
scene_ids_of_confirmed_markers = list(set(scene_ids_of_confirmed_markers))

In [ ]:
scenes = []

for scene_id in scene_ids_of_confirmed_markers:
    scene = stash.find_scene(scene_id, fragment="id title")
    scenes.append(scene)

In [ ]:
filter = {
    "tags": {
        "value": [stash_tags.ai_reviewed["id"]],
        "modifier": "INCLUDES_ALL",
    }
}
scene_ids = [scene["id"] for scene in scenes]
result = stash.call_GQL("""query FindScenes($scene_filter: SceneFilterType, $scene_ids: [Int!]) {
    findScenes(scene_filter: $scene_filter, scene_ids: $scene_ids) {
        count
        scenes {
            id
            title
            performers {
                id
                name
            }
        }
    }
}""", variables={"scene_filter": filter, "scene_ids": scene_ids})
result